We might need to use the Spark’s lower-level APIs, specifically the Resilient Distributed Dataset (RDD), the SparkContext, and distributed shared variables like accumulators and broadcast variables.


There are two sets of low-level APIs: there is one for manipulating distributed data (RDDs), and another for distributing and manipulating distributed shared variables (broadcast variables and accumulators).


A SparkContext is the entry point for low-level API functionality. You access it through the SparkSession


### Datasets and RDDs of Case Classes
We noticed this question on the web and found it to be an interesting one: what is the difference between RDDs of Case Classes and Datasets? The difference is that Datasets can still take advantage of the wealth of functions and optimizations that the Structured APIs have to offer. With Datasets, you do not need to choose between only operating on JVM types or on Spark types, you can choose whatever is either easiest to do or most flexible. You get the both of best worlds.

In [1]:
spark.sparkContext

Intitializing Scala interpreter ...

Spark Web UI available at http://kindis-mbp:4044
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1646844762948)
SparkSession available as 'spark'


res0: org.apache.spark.SparkContext = org.apache.spark.SparkContext@4a5f82ff


## Creating an RDD
One of the easiest ways to get RDDs is from an existing DataFrame or Dataset. Converting these to an RDD is simple: just use the `rdd` method on any of these data types.

In [3]:
// in Scala: converts a Dataset[Long] to RDD[Long]
val valRdd = spark.range(500).rdd

valRdd: org.apache.spark.rdd.RDD[Long] = MapPartitionsRDD[11] at rdd at <console>:25


An RDD of type Row is a DataFrame

In [6]:
spark.range(10).toDF().collect()

res3: Array[org.apache.spark.sql.Row] = Array([0], [1], [2], [3], [4], [5], [6], [7], [8], [9])


In [9]:
val d= spark.range(10).toDF()

d: org.apache.spark.sql.DataFrame = [id: bigint]


In [15]:
d.dtypes

res9: Array[(String, String)] = Array((id,LongType))


In [8]:
// in Scala
//spark.range(10).toDF().rdd.map(rowObject => rowObject.getLong(0)).collect()

### From a Local Collection
To create an RDD from a collection, you will need to use the `parallelize` method on a `SparkContext` (within a `SparkSession`). This turns a single node collection into a parallel collection.

In [36]:
// in Scala
val myCollection = "Spark The Definitive Guide : Big Data Processing Made Simple, yeah that is the truth about Data"
  .split(" ")

val words = spark.sparkContext.parallelize(myCollection, 2)

myCollection: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple,, yeah, that, is, the, truth, about, Data)
words: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[52] at parallelize at <console>:30


In [37]:
words

res28: org.apache.spark.rdd.RDD[String] = ParallelCollectionRDD[52] at parallelize at <console>:30


An additional feature is that you can then name this RDD to show up in the Spark UI according to a given name:

In [38]:
// in Scala
words.setName("myWords")
words.name // myWords

res29: String = myWords


### From Data Sources
RDDs do not have a notion of “Data Source APIs” like DataFrames do; they primarily define their dependency structures and lists of partitions.

In [28]:
val rdd=spark.sparkContext.textFile("scala-spark-tutorial/in/airports.text")

rdd: org.apache.spark.rdd.RDD[String] = scala-spark-tutorial/in/airports.text MapPartitionsRDD[36] at textFile at <console>:24


In [29]:
rdd.collect()

res21: Array[String] = Array(1,"Goroka","Goroka","Papua New Guinea","GKA","AYGA",-6.081689,145.391881,5282,10,"U","Pacific/Port_Moresby", 2,"Madang","Madang","Papua New Guinea","MAG","AYMD",-5.207083,145.7887,20,10,"U","Pacific/Port_Moresby", 3,"Mount Hagen","Mount Hagen","Papua New Guinea","HGU","AYMH",-5.826789,144.295861,5388,10,"U","Pacific/Port_Moresby", 4,"Nadzab","Nadzab","Papua New Guinea","LAE","AYNZ",-6.569828,146.726242,239,10,"U","Pacific/Port_Moresby", 5,"Port Moresby Jacksons Intl","Port Moresby","Papua New Guinea","POM","AYPY",-9.443383,147.22005,146,10,"U","Pacific/Port_Moresby", 6,"Wewak Intl","Wewak","Papua New Guinea","WWK","AYWK",-3.583828,143.669186,19,10,"U","Pacific/Port_Moresby", 7,"Narsarsuaq","Narssarssuaq","Greenland","UAK","BGBW",61.160517,-45.425978,112,-3,"...


In [30]:
// alternative to read each file as a records
//spark.sparkContext.wholeTextFiles("/some/path/withTextFiles")

## Manipulating RDDs
You manipulate RDDs in much the same way that you manipulate DataFrames. As mentioned, the core difference being that you manipulate `raw Java` or `Scala objects` instead of `Spark types`


### Transformations

#### distinct
A distinct method call on an RDD removes duplicates from the RDD:

In [39]:
println(words.count())
words.distinct().count()

17


res30: Long = 16


### filter
Filtering is equivalent to creating a SQL-like where clause. You can look through our records in the RDD and see which ones match some predicate function.

In [40]:
// in Scala
def startsWithS(individual:String) = {
  individual.startsWith("S")
}

startsWithS: (individual: String)Boolean


In [42]:
words.filter(startsWithS).collect()

res32: Array[String] = Array(Spark, Simple,)


In [43]:
// in Scala
words.filter(word => startsWithS(word)).collect()

res33: Array[String] = Array(Spark, Simple,)


### Map
You specify a function that returns the value that you want, given the correct input. You then apply that, record by record. Let’s perform something similar to what we just did. In this example, we’ll map the current word to the word, its starting letter, and whether the word begins with “S.”

In [44]:
// in Scala
val words2 = words.map(word => (word, word(0), word.startsWith("S")))

words2: org.apache.spark.rdd.RDD[(String, Char, Boolean)] = MapPartitionsRDD[59] at map at <console>:27


In [45]:
words2.collect()

res34: Array[(String, Char, Boolean)] = Array((Spark,S,true), (The,T,false), (Definitive,D,false), (Guide,G,false), (:,:,false), (Big,B,false), (Data,D,false), (Processing,P,false), (Made,M,false), (Simple,,S,true), (yeah,y,false), (that,t,false), (is,i,false), (the,t,false), (truth,t,false), (about,a,false), (Data,D,false))


In [46]:
// filter relevant boolean values
// in Scala
words2.filter(record => record._3).take(5)

res35: Array[(String, Char, Boolean)] = Array((Spark,S,true), (Simple,,S,true))


### flatMap
flatMap provides a simple extension of the `map` function we just looked at. Sometimes, each current row should return multiple rows, instead. 

For example, you might want to take your set of words and `flatMap` it into a set of characters. Because each word has multiple characters, you should use flatMap to expand it. flatMap requires that the ouput of the map function be an iterable that can be expanded:

In [51]:
words.collect()

res40: Array[String] = Array(Spark, The, Definitive, Guide, :, Big, Data, Processing, Made, Simple,, yeah, that, is, the, truth, about, Data)


In [53]:
// in Scala
words.flatMap(word => word.toSeq).take(10)

res42: Array[Char] = Array(S, p, a, r, k, T, h, e, D, e)


### Random Splits
We can also randomly split an RDD into an Array of RDDs by using the randomSplit method, which accepts an Array of weights and a random seed:

In [54]:
// in Scala
val fiftyFiftySplit = words.randomSplit(Array[Double](0.5, 0.5))

fiftyFiftySplit: Array[org.apache.spark.rdd.RDD[String]] = Array(MapPartitionsRDD[67] at randomSplit at <console>:27, MapPartitionsRDD[68] at randomSplit at <console>:27)


In [60]:
fiftyFiftySplit.take(3)

res48: Array[org.apache.spark.rdd.RDD[String]] = Array(MapPartitionsRDD[67] at randomSplit at <console>:27, MapPartitionsRDD[68] at randomSplit at <console>:27)


## Actions
Just as we do with DataFrames and Datasets, we specify actions to kick off our specified transformations. 

Actions either collect data to the `driver` or `write` to an external data source.

### reduce
You can use the reduce method to specify a function to “reduce” an RDD of any kind of value to one value. For instance, given a set of numbers, you can reduce this to its sum by specifying a function that takes as input two values and reduces them into one. 

In [61]:
// in Scala
spark.sparkContext.parallelize(1 to 20).reduce(_ + _) // 210

res49: Int = 210


You can also use this to get something like the longest word in our set of words that we defined a moment ago. The key is just to define the correct function:

In [62]:
// in Scala
def wordLengthReducer(leftWord:String, rightWord:String): String = {
  if (leftWord.length > rightWord.length)
    return leftWord
  else
    return rightWord
}

words.reduce(wordLengthReducer)

wordLengthReducer: (leftWord: String, rightWord: String)String
res50: String = Processing


This reducer is a good example because you can get one of two outputs. Because the reduce operation on the partitions is not deterministic, you can have either “definitive” or “processing” (both of length 10) as the “left” word. This means that sometimes you can end up with one, whereas other times you end up with the other.

### count
This method is fairly self-explanatory. Using it, you could, for example, count the number of rows in the RDD:

In [63]:
words.count()

res51: Long = 17


### countByValue
This method counts the number of values in a given RDD. However, it does so by finally loading the result set into the memory of the driver. You should use this method only if the resulting map is expected to be small because the entire thing is loaded into the driver’s memory. Thus, this method makes sense only in a scenario in which either the total number of rows is low or the number of distinct items is low:

In [64]:
words.countByValue()

res52: scala.collection.Map[String,Long] = Map(Definitive -> 1, is -> 1, Processing -> 1, that -> 1, The -> 1, yeah -> 1, Spark -> 1, Made -> 1, Guide -> 1, Big -> 1, : -> 1, Simple, -> 1, truth -> 1, about -> 1, Data -> 2, the -> 1)


### first
The `first` method returns the first value in the dataset:

In [65]:
words.first()

res53: String = Spark


### max and min
max and min return the maximum values and minimum values, respectively:

In [68]:
spark.sparkContext.parallelize(1 to 20).max()

res56: Int = 20


In [67]:
spark.sparkContext.parallelize(1 to 20).min()

res55: Int = 1


### take
`take` and its derivative methods take a number of values from your RDD. This works by first scanning one partition and then using the results from that partition to estimate the number of additional partitions needed to satisfy the limit.

There are many variations on this function, such as `takeOrdered, takeSample, and top`. You can use `takeSample` to specify a fixed-size random sample from your RDD. You can specify whether this should be done by using `withReplacement`, the number of values, as well as the random seed. top is effectively the opposite of `takeOrdered` in that it selects the top values according to the implicit ordering:

In [70]:
words.take(5)
words.takeOrdered(5)
words.top(5)
val withReplacement = false //true
val numberToTake = 6
val randomSeed = 100L
words.takeSample(withReplacement, numberToTake, randomSeed)

withReplacement: Boolean = false
numberToTake: Int = 6
randomSeed: Long = 100
res58: Array[String] = Array(Processing, Simple,, Spark, that, Data, Data)


## Saving Files
Saving files means writing to plain-text files. With RDDs, you cannot actually “save” to a data source in the conventional sense. You must iterate over the partitions in order to save the contents of each partition to some external database. This is a low-level approach that reveals the underlying operation that is being performed in the higher-level APIs. Spark will take each partition, and write that out to the destination.

### saveAsTextFile
To save to a text file, you just specify a path and optionally a compression codec:



In [ ]:
words.saveAsTextFile("file:/tmp/bookTitle")

To set a compression codec, we must import the proper codec from Hadoop. You can find these in the `org.apache.hadoop.io.compress` library:

In [71]:
// in Scala
import org.apache.hadoop.io.compress.BZip2Codec
words.saveAsTextFile("file:/tmp/bookTitleCompressed", classOf[BZip2Codec])

import org.apache.hadoop.io.compress.BZip2Codec


### SequenceFiles
Spark originally grew out of the Hadoop ecosystem, so it has a fairly tight integration with a variety of Hadoop tools. A `sequenceFile` is a flat file consisting of `binary key–value` pairs. It is extensively used in `MapReduce as input/output formats`.

Spark can write to sequenceFiles using the `saveAsObjectFile` method or by explicitly writing key–value pairs

In [72]:
words.saveAsObjectFile("/tmp/my/sequenceFilePath")

## Checkpointing
One feature not available in the DataFrame API is the concept of checkpointing. Checkpointing is the act of saving an RDD to disk so that future references to this RDD point to those intermediate partitions on disk rather than recomputing the RDD from its original source. This is similar to caching except that it’s not stored in memory, only disk. This can be helpful when performing iterative computation, similar to the use cases for caching:

In [ ]:
spark.sparkContext.setCheckpointDir("/some/path/for/checkpointing")
words.checkpoint()

Now, when we reference this RDD, it will derive from the checkpoint instead of the source data. This can be a helpful optimization.

## Pipe RDDs to System Commands
The pipe method is probably one of Spark’s more interesting methods. With pipe, you can return an RDD created by piping elements to a forked external process. The resulting RDD is computed by executing the given process once per partition

In [74]:
words.pipe("wc -l").collect()

res62: Array[String] = Array("       8", "       9")


### mapPartitions
The previous command revealed that Spark operates on a per-partition basis when it comes to actually executing code. 

You also might have noticed earlier that the return signature of a map function on an RDD is actually MapPartitionsRDD. This is because map is just a row-wise alias for mapPartitions, which makes it possible for you to map an individual partition (represented as an iterator). 


That’s because physically on the cluster we operate on each partition individually (and not a specific row). A simple example creates the value “1” for every partition in our data, and the sum of the following expression will count the number of partitions we have

In [75]:
// in Scala
words.mapPartitions(part => Iterator[Int](1)).sum() // 2

res63: Double = 2.0


Other functions similar to mapPartitions include mapPartitionsWithIndex. With this you specify a function that accepts an index (within the partition) and an iterator that goes through all items within the partition. The partition index is the partition number in your RDD, which identifies where each record in our dataset sits (and potentially allows you to debug). You might use this to test whether your map functions are behaving correctly:



In [76]:
// in Scala
def indexedFunc(partitionIndex:Int, withinPartIterator: Iterator[String]) = {
  withinPartIterator.toList.map(
    value => s"Partition: $partitionIndex => $value").iterator
}
words.mapPartitionsWithIndex(indexedFunc).collect()

indexedFunc: (partitionIndex: Int, withinPartIterator: Iterator[String])Iterator[String]
res64: Array[String] = Array(Partition: 0 => Spark, Partition: 0 => The, Partition: 0 => Definitive, Partition: 0 => Guide, Partition: 0 => :, Partition: 0 => Big, Partition: 0 => Data, Partition: 0 => Processing, Partition: 1 => Made, Partition: 1 => Simple,, Partition: 1 => yeah, Partition: 1 => that, Partition: 1 => is, Partition: 1 => the, Partition: 1 => truth, Partition: 1 => about, Partition: 1 => Data)


## foreachPartition
Although mapPartitions needs a return value to work properly, this next function does not. foreachPartition simply iterates over all the partitions of the data. The difference is that the function has no return value. This makes it great for doing something with each partition like writing it out to a database. In fact, this is how many data source connectors are written. You can create our own text file source if you want by specifying outputs to the temp directory with a random ID

In [77]:
words.foreachPartition { iter =>
  import java.io._
  import scala.util.Random
  val randomFileName = new Random().nextInt()
  val pw = new PrintWriter(new File(s"/tmp/random-file-${randomFileName}.txt"))
  while (iter.hasNext) {
      pw.write(iter.next())
  }
  pw.close()
}

## glom
glom is an interesting function that takes every partition in your dataset and converts them to arrays. This can be useful if you’re going to collect the data to the driver and want to have an array for each partition. However, this can cause serious stability issues because if you have large partitions or a large number of partitions, it’s simple to crash the driver.

In [78]:
// in Scala
spark.sparkContext.parallelize(Seq("Hello", "World"), 2).glom().collect()
// Array(Array(Hello), Array(World))


res66: Array[Array[String]] = Array(Array(Hello), Array(World))
